## Notebook Instructions

1. If you are new to Jupyter notebooks, please go through this introductory manual <a href='https://quantra.quantinsti.com/quantra-notebook' target="_blank">here</a>.
1. Any changes made in this notebook would be lost after you close the browser window. **You can download the notebook to save your work on your PC.**
1. Before running this notebook on your local PC:<br>
i.  You need to set up a Python environment and the relevant packages on your local PC. To do so, go through the section on "**Run Codes Locally on Your Machine**" in the course.<br>
ii. You need to **download the zip file available in the last unit** of this course. The zip file contains the data files and/or python modules that might be required to run this notebook.

# Calculate Order Flow With Tick Rule

In the previous unit, we discussed how to calculate the order flow using the tick rule. The tick rule uses the current trade price, previous trade price and trade volume to calculate the order flow. In this notebook, you will learn to implement the tick rule for order flow calculation. 

The notebook is structured as follows:
1. [Read the Data](#data)
1. [Calculate the Tick Direction](#tick_direction)
1. [Calculate the Order Flow](#order_flow)

## Import Libraries

In [1]:
# For data manipulation
import pandas as pd
import numpy as np

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

<a id='data'></a>
## Read the Data
Import the pickle file `tick_data_es500_2013_10_02.bz2` as the dataframe `tick_data`. This pickle file has tick data and best bid, ask data of 	E-mini S&P 500 futures for the date `2013-10-02`. This dataset has columns `bid`, `ask`, `trade_price` and `trade_size` with datetime in milliseconds as the index.

**Note:** _The python version 3.9.5 and pandas version 1.4.4 were used to create this pickle file. To avoid issues reading this pickle file, ensure that the python and pandas versions are equal or higher than the versions mentioned._

In [2]:
# Read the 'tick_data.pkl' pickle file as 'tick_data' dataframe
tick_data = pd.read_pickle('../data_modules/tick_data_es500_2013_10_02.bz2')

# Print first 5 trades of the 'tick_data' dataframe
tick_data.head()

,bid,ask,trade_price,trade_size
,,,,
2013-10-02 00:00:00.604800,1428.50,1428.75,1428.75,1.0
2013-10-02 00:00:10.972800,1428.50,1428.75,1428.75,25.0
2013-10-02 00:00:19.439999,1428.50,1428.75,1429.00,17.0
2013-10-02 00:00:19.439999,1428.50,1428.75,1429.00,11.0
2013-10-02 00:00:19.526400,1428.75,1429.00,1429.00,28.0


<a id='tick_direction'></a>
## Calculate the Tick Direction

The first step is to find if the trade resulted in an up-tick or a down-tick. This information is commonly known as the tick direction.

To calculate the tick direction, we need to compute the difference between the current trade price and the previous trade price. 

![tick_rule.jpg](https://d2a032ejo53cab.cloudfront.net/Glossary/wv0JqLmc/tickrule-1.jpg)
 

* Calculate the tick direction by finding the difference between the current trade price and the previous trade price using the `diff()` method of pandas. 
* Use the `np.sign` function to obtain the sign of the difference. Finally, store the result in a new column called `tick_direction` in the `tick_data` dataframe.

The tick direction of +1 indicates an up-tick and -1 indicates a down-tick. The value of tick direction would be 0 if the current trade price is the same as the previous trade price. This indicates that the tick direction is undefined.

In [3]:
# Store the tick direction of each trade in the column 'trade_direction'
tick_data['tick_direction'] = tick_data['trade_price'].diff().apply(np.sign)

# Print first 5 trades of the 'tick_data' dataframe
tick_data.head()

,bid,ask,trade_price,trade_size,tick_direction
,,,,,
2013-10-02 00:00:00.604800,1428.50,1428.75,1428.75,1.0,NaN
2013-10-02 00:00:10.972800,1428.50,1428.75,1428.75,25.0,0.0
2013-10-02 00:00:19.439999,1428.50,1428.75,1429.00,17.0,1.0
2013-10-02 00:00:19.439999,1428.50,1428.75,1429.00,11.0,0.0
2013-10-02 00:00:19.526400,1428.75,1429.00,1429.00,28.0,0.0


As you can see, the tick direction of the first trade is `NaN` since there was no previous trade to the very first trade. In this case, you can choose to exclude the first trade from the order flow calculations.


In [4]:
# Excluding first trade from the order flow calculations
tick_data = tick_data[1:]

# Print first 5 trades of the 'tick_data' dataframe
tick_data.head()

,bid,ask,trade_price,trade_size,tick_direction
,,,,,
2013-10-02 00:00:10.972800,1428.50,1428.75,1428.75,25.0,0.0
2013-10-02 00:00:19.439999,1428.50,1428.75,1429.00,17.0,1.0
2013-10-02 00:00:19.439999,1428.50,1428.75,1429.00,11.0,0.0
2013-10-02 00:00:19.526400,1428.75,1429.00,1429.00,28.0,0.0
2013-10-02 00:00:19.612800,1428.75,1429.00,1429.00,3.0,0.0


Before we move to the final step of order flow calculation, we need to handle the trades for which the tick direction is undefined i.e. `tick_direction` is 0. 

We can use the forward-fill (`ffill`) method to replace the value of 0 in the `tick_direction` column with the previous non-zero tick direction value.

In [5]:
# Replace all 0 values in the 'tick_direction' column
# with the previous non-zero value in the same column
tick_data['tick_direction'] = tick_data['tick_direction'].replace(
    0, method='ffill')

# Print first 5 trades of the 'tick_data' dataframe
tick_data.head()

,bid,ask,trade_price,trade_size,tick_direction
,,,,,
2013-10-02 00:00:10.972800,1428.50,1428.75,1428.75,25.0,0.0
2013-10-02 00:00:19.439999,1428.50,1428.75,1429.00,17.0,1.0
2013-10-02 00:00:19.439999,1428.50,1428.75,1429.00,11.0,1.0
2013-10-02 00:00:19.526400,1428.75,1429.00,1429.00,28.0,1.0
2013-10-02 00:00:19.612800,1428.75,1429.00,1429.00,3.0,1.0


As you can see, the tick direction of the first trade is still 0 since it has no previous trades to replace the 0 value of the tick direction. For accurate order flow calculation, we will remove the first trade since its tick direction is undefined.

In [6]:
# Excluding first trade from the order flow calculations since it's tick direction is undefined
tick_data = tick_data[1:]

<a id='order_flow'></a>
## Calculate the Order Flow

According to the definition, the order flow is the signed transaction volume. 
So, as per the tick rule, 
* The order flow is equal to the volume of the trade with a positive sign if the trade resulted in an up-tick. <br>
* Similarly, if the trade resulted in a down-tick, then the order flow is equal to the volume of the trade with a negative sign.<br>

To calculate the order flow of each trade, we will multiply the column `trade_size` with the column `tick_direction`. Finally, the result is stored in a new column called `order_flow` in the `tick_data` dataframe.

In [7]:
# Calculate the order flow using tick rule
tick_data['order_flow'] = tick_data['tick_direction'] * tick_data['trade_size']

# Print first 5 trades of the 'tick_data' dataframe
tick_data.head()

,bid,ask,trade_price,trade_size,tick_direction,order_flow
,,,,,,
2013-10-02 00:00:19.439999,1428.50,1428.75,1429.0,17.0,1.0,17.0
2013-10-02 00:00:19.439999,1428.50,1428.75,1429.0,11.0,1.0,11.0
2013-10-02 00:00:19.526400,1428.75,1429.00,1429.0,28.0,1.0,28.0
2013-10-02 00:00:19.612800,1428.75,1429.00,1429.0,3.0,1.0,3.0
2013-10-02 00:00:24.019199,1428.75,1429.00,1429.0,5.0,1.0,5.0


## Conclusion

In this notebook, you learnt to calculate the order flow using the tick rule. The order flow can be used as an indicator of price momentum and a good predictor of price direction. In the next unit, you will learn to calculate the order flow using the quote rule and lee-ready algorithm.<br><br>